## 공용 코드

In [1]:
# 파이썬
# ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 공통 모듈 임포트
import numpy as np
import pandas as pd
import os

path = "c:/Windows/Fonts/malgun.ttf"

from matplotlib import font_manager, rc

import platform

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
    
# Jupyter Notebook의 출력을 소수점 이하 3자리로 제한
%precision 3

import sklearn
assert sklearn.__version__ >= "0.20"

# 시드 고정
np.random.seed(21)

## 데이터

### 데이터 가져오기

In [2]:
origin_data = pd.read_excel('./3차_VOD_2308.xlsx', index_col = None)

### 데이터 전처리

In [3]:
# 불필요한 컬럼 제거
drop_data = origin_data.drop(['Unnamed: 0', 'ct_cl', 'genre_of_ct_cl', 'use_tms', 'SMRY', 'disp_rtm', 'strt_dt'], axis = 1)

In [4]:
# 데이터에서 불필요한 부분 제거

import re

# 제목 데이터에서 불필요한 부분을 제거하고
# 전처리된 제목을 새로운 컬럼으로 추가하는 함수
def get_rid_asset(dataframe):
    # 괄호 안의 불필요한 부분 패턴
    pattern = re.compile('\([A-Za-z가-힣\s]+\)|\([가-힣\d\s]+\)|\([\d\/]{5,}\)')

    new_data = []

    for item in dataframe['asset_nm']:
        name = str(item)
        match_pattern = pattern.findall(name)

        for dt in match_pattern:
            name = name.replace(dt, "")
            #new_data.append(dt)

        new_data.append(name) 
    
    # 제목과 함께 존재하는 회차 정보 제거
    pattern = re.compile('\s[\d]+회')

    series = []

    for item in new_data:
        name = str(item)

        # 정규 표현식을 만족하는 경우 - 드라마 컨텐츠
        try:
            match_pattern = pattern.findall(name)

            for dt in match_pattern:
                name = name.replace(dt, "")
            series.append(name)

        # 정규 표현식을 만족하지 않는 경우 - 영화(~~회 가 없음)
        except:
            series.append(item)
    
    # 데이터 프레임에 처리 결과 컬럼을 추가
    dataframe['series_name'] = series

In [5]:
# 데이터에 함수를 적용
get_rid_asset(drop_data)

# 함수 적용 결과 확인
#print(drop_data.head())

In [6]:
# 출연진 정보가 없는 데이터 제거
no_actr_data = drop_data.drop(drop_data[drop_data['ACTR_DISP'] == '-'].index, axis = 0)

# 제거 확인 - 출력되는 데이터 없음
# print(no_actr_data[no_actr_data['ACTR_DISP'] == '-'])

In [ ]:
drop_data.head()

In [ ]:
# 특정 subsr 의 시청 데이터에서 출연진 가져오기
view_data = no_actr_data[drop_data['subsr'] == 64385000]

# 데이터 확인
print(view_data)
print(view_data.groupby('series_name').count())

## 출연진 기반 추천

### 출연진 데이터

In [36]:
# 출연진 정보를 저장할 list
actr_list = []

# 시청 데이터를 순회하면서 출연진 정보를 저장
for _, item in view_data.iterrows():
    if ',' in item['ACTR_DISP']:
        item_list = item['ACTR_DISP'].split(',')
    else:
        item_list = [item['ACTR_DISP']]
    
    for actr in item_list:
        actr_list.append(actr)

# 확인
#print(actr_list)

In [37]:
# counter 를 사용해서 출연 횟수 확인
from collections import Counter

count = Counter(actr_list)

# 가장 많이 출연한 상위 N 명의 출연진 정보
N_people = 7
bestN_actr = count.most_common(N_people)

In [38]:
# 상위 N위 출연진에 대해 가중치 설정
score_sum = 0
for actr_score in bestN_actr:
    score_sum += actr_score[1]

w_bestN = []
for actr_score in bestN_actr:
    w_bestN.append([actr_score[0], round((actr_score[1] / score_sum), 2)])
    
print(w_bestN)

[['신동엽', 0.17], ['정선희', 0.17], ['토니안 ', 0.17], ['조이', 0.17], ['이연복', 0.11], ['이정현', 0.11], ['이유리', 0.11]]


In [ ]:
actr_score_list = []

# 각 행을 순회하면서 출연진 정보에 대한 점수를 더해서 데이터 생성
for _, item in drop_data.iterrows():
    score_data = 0
    
    for name_score in w_bestN:
        if name_score[0] in str(item['ACTR_DISP']):
            score_data += name_score[1]
    actr_score_list.append(score_data)
    
# 기존 데이터에 컬럼 추가
drop_data['ATCR_SCORE'] = actr_score_list

# 확인
drop_data.head()

In [40]:
# 출연진 점수를 기준으로 순위 부여
drop_data['rank'] = drop_data['ATCR_SCORE'].rank(method = 'min', na_option = 'bottom',
                                                    ascending = False)
# 확인
drop_data['ATCR_SCORE'].value_counts()

0.00    5318
0.68     118
0.17      79
0.33      61
0.11      39
Name: ATCR_SCORE, dtype: int64

In [ ]:
drop_data[drop_data['ATCR_SCORE'] == 0.68]

In [55]:
drop_data[drop_data['ATCR_SCORE'] == 0.33].groupby('series_name').count()

,subsr,asset_nm,ACTR_DISP,ATCR_SCORE,rank
series_name,,,,,
신상 출시 편스토랑,61,61,61,61,61


In [ ]:
drop_data[drop_data['ATCR_SCORE'] == 0.17]

In [ ]:
drop_data[drop_data['ATCR_SCORE'] == 0.17].groupby('series_name').count()

In [33]:
(origin_data.groupby(['subsr']).count())[(origin_data.groupby(['subsr']).count())['asset_nm'] == 373]

,Unnamed: 0,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
subsr,,,,,,,,,
64385000,373,373,373,373,373,373,373,373,373


In [ ]:
print(view_data.groupby('series_name').count())

In [ ]:
# ACTR score 를 기반으로 상위 컨텐츠 출력

# 불필요한 컬럼 제거
ac_data = drop_data.drop(['subsr', 'asset_nm', 'ACTR_DISP'], axis = 1)
#print(ac_data.head())

# series_name 과 ATCR_SCORE 컬럼을 기준으로 묶어서 중복 제거
count_data = ac_data.groupby(['series_name', 'ATCR_SCORE']).count().reset_index()
#print(count_data.head())

# ATCR_SCORE 를 기준으로 정렬을 수행
sorted_ac_data = count_data.sort_values(by = ['ATCR_SCORE', 'series_name'], ascending = False)
#print(type(sorted_ac_data))
#print(sorted_ac_data.head())

# 최종 정렬 결과 확인
print(sorted_ac_data)

In [111]:
# 상위 5개 컨텐츠의 정보를 리스트로 생성
top5 = list(sorted_ac_data['series_name'][:5])
print(top5)

['TV동물농장', '신상 출시 편스토랑', '익스토션', '실화탐사대', '슈퍼 마리오 브라더스']
